In [1]:
!pip install scikit-learn

In [2]:
import pandas as pd
import torch
from transformers import BartTokenizer, BartForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset
from sklearn.model_selection import train_test_split

In [3]:
df = pd.read_csv("../data/processed/PoetryFoundationData_Cleaned.csv")

In [4]:
train_df, val_df = train_test_split (df, test_size = 0.1, random_state= 42)
train_dataset = Dataset.from_pandas(train_df)
val_dataset = Dataset.from_pandas(val_df)

In [5]:
tokenizer = BartTokenizer.from_pretrained("facebook/bart-base")

In [14]:
def preprocess_function(examples):
    inputs = examples["Title"]
    targets = examples["Poem"]
    model_inputs = tokenizer(
        inputs, 
        max_length=64, 
        truncation=True, 
        padding="max_length"
    )
    with tokenizer.as_target_tokenizer():
        labels = tokenizer(
            targets, 
            max_length=256, 
            truncation=True, 
            padding="max_length"
        )
    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_train = train_dataset.map(preprocess_function, batched=True)
tokenized_val = val_dataset.map(preprocess_function, batched = True)

Map:   0%|          | 0/12306 [00:00<?, ? examples/s]

C:\Users\Ajit\anaconda3\envs\poetry-nlp\lib\site-packages\transformers\tokenization_utils_base.py:4034: UserWarning: `as_target_tokenizer` is deprecated and will be removed in v5 of Transformers. You can tokenize your labels by using the argument `text_target` of the regular `__call__` method (either in the same call as your input texts if you use the same keyword arguments, or in a separate call.
  warnings.warn(
Exception in thread Thread-5:
Traceback (most recent call last):
  File "C:\Users\Ajit\anaconda3\envs\poetry-nlp\lib\threading.py", line 1016, in _bootstrap_inner
    self.run()
  File "C:\Users\Ajit\anaconda3\envs\poetry-nlp\lib\site-packages\tqdm\_monitor.py", line 84, in run
    instance.refresh(nolock=True)
  File "C:\Users\Ajit\anaconda3\envs\poetry-nlp\lib\site-packages\tqdm\std.py", line 1347, in refresh
    self.display()
  File "C:\Users\Ajit\anaconda3\envs\poetry-nlp\lib\site-packages\tqdm\notebook.py", line 171, in display
    rtext.value = right
  File "C:\Users\A

Map:   0%|          | 0/1368 [00:00<?, ? examples/s]

In [15]:
model = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

In [16]:
!pip install --upgrade transformers

In [17]:
pip install transformers[torch]

In [18]:
training_args = TrainingArguments(
    output_dir="./experiments",
    logging_strategy="steps",
    learning_rate=2e-5,
    per_device_train_batch_size=4,
    per_device_eval_batch_size=4,
    num_train_epochs=3,
    weight_decay=0.01,
    logging_dir="./logs",
    save_total_limit=2,
    save_strategy="epoch"
)

In [19]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_train,
    eval_dataset=tokenized_val,
)

In [20]:
trainer.train()

Step,Training Loss
500,3.779500


KeyboardInterrupt: 

In [ ]:
inputs = tokenizer("The night is dark and full of", return_tensors="pt")
outputs = model.generate(**inputs, max_new_tokens=50)
print(tokenizer.decode(outputs[0], skip_special_tokens=True))

In [ ]:
model.save_pretrained("models/fine_tuned_bart_poetry")
tokenizer.save_pretrained("models/fine_tuned_bart_poetry")